In [ ]:
from sklearn.ensemble import IsolationForest
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder
from sklearn.decomposition import PCA
from sklearn.impute import KNNImputer
import numpy as np
import requests
import os

# Creating user login data
data = pd.DataFrame(columns=['login_time', 'location'])
rows = []
for i in range(1000):
    rows.append({
        'login_time': np.random.randint(1, 1000),
        'location': np.random.choice(['US', 'FR', 'DE', 'IN', 'UK', 'AU', 'CA'])
    })
    
data = pd.DataFrame(rows)
print(data.head(10))
data['location'] = LabelEncoder().fit_transform(data['location'])
imputer = KNNImputer(n_neighbors=2)
imputed_data = pd.DataFrame(imputer.fit_transform(data))
imputed_data.columns = data.columns

# Isolation Forest Model
model = IsolationForest(contamination=0.2, n_jobs=-1)  # Utilize all cores
model.fit(imputed_data)
anomalies = model.predict(imputed_data)
plt.scatter(imputed_data['login_time'], imputed_data['location'], c=anomalies, cmap='coolwarm', alpha=0.6)

anomaly_data = imputed_data[anomalies == -1]
plt.scatter(anomaly_data['login_time'], anomaly_data['location'], c='red', marker='x', s=200)
plt.xlabel('Login Time')
plt.ylabel('Location')
plt.title('Anomaly Detection')
plt.show()

ip_file = 'ip_addresses.txt'
if not os.path.exists(ip_file):
    open(ip_file, 'w').close()
current_ip = requests.get('https://api.ipify.org').text

print("Anomaly Data:")
for index, row in anomaly_data.iterrows():
    with open(ip_file, 'a') as f:
        f.write(current_ip + '\n')
    # Read the IP addresses from the file
    with open(ip_file, 'r') as f:
        ip_addresses = f.readlines()

    # Printing the IP addresses
    #print(f"Login Time: {row['login_time']}, Location: {data['location'][index]}, Current IP: {current_ip}, Previous IPs: {[ip.strip() for ip in ip_addresses]}")

# Using PCA to reduce dimensionality
pca = PCA(n_components=2)
pca_data = pca.fit_transform(imputed_data)
plt.scatter(pca_data[:, 0], pca_data[:, 1], c=anomalies, cmap='coolwarm', alpha=0.6)
mask = anomalies == -1
plt.scatter(pca_data[mask, 0], pca_data[mask, 1], c='red', marker='x', s=200)
plt.xlabel('Principal Component 1')
plt.ylabel('Principal Component 2')
plt.title('Anomaly Detection with PCA')
plt.show()
